In [1]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
MODEL_NAME = "distilbert-base-uncased"   # or bert-base-uncased
MODEL_PATH = "C:/Users/parth/Desktop/phising_Website Detection/best_model.pt"             # your saved .pt file

URL_CSV = "C:/Users/parth/Desktop/phising_Website Detection/Phish_URL.csv"
EMAIL_CSV = "C:/Users/parth/Desktop/phising_Website Detection/Phishing_Email.csv"

MAX_LEN = 128
BATCH_SIZE = 8

In [3]:
def load_url_dataset():
    df = pd.read_csv(URL_CSV)
    data = pd.DataFrame()
    data["text"] = df["URL"].astype(str)
    data["label"] = df["label"].astype(int)
    return data

In [4]:
def load_email_dataset():
    df = pd.read_csv(EMAIL_CSV)
    data = pd.DataFrame()
    data["text"] = df["Email Text"].astype(str)
    data["label"] = df["Email Type"].apply(
        lambda x: 1 if "phish" in x.lower() else 0
    )
    return data

In [5]:
df_url = load_url_dataset()
df_email = load_email_dataset()

df = pd.concat([df_url, df_email], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

texts = df["text"].tolist()
labels = df["label"].tolist()

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EvalDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            max_length=MAX_LEN,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

eval_ds = EvalDataset(texts, labels)
eval_loader = DataLoader(eval_ds, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

print("Model loaded successfully.")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully.


In [9]:
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=ids, attention_mask=mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

C:\Users\parth\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:392: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [10]:
print("Accuracy:", accuracy_score(all_labels, all_preds))

print("\nClassification Report:\n")
print(classification_report(
    all_labels,
    all_preds,
    target_names=["Legitimate", "Phishing"]
))

Accuracy: 0.9969345045098155

Classification Report:

              precision    recall  f1-score   support

  Legitimate       1.00      0.99      1.00    112267
    Phishing       1.00      1.00      1.00    142178

    accuracy                           1.00    254445
   macro avg       1.00      1.00      1.00    254445
weighted avg       1.00      1.00      1.00    254445

